In [82]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
import fiona
from shapely.ops import unary_union
import networkx as nx
import itertools
import osmnx as ox
import math
import momepy
import matplotlib.pyplot as plt
from shapely.geometry import Point
import io

In [83]:
# import SF Road_network
road_network=gpd.read_file('H:/mmmiah/Desktop/Bicycle Connectivity Project/SF Validation_data/LTS_data/Speed_aadt.shp')
road_network.head()

,cnn,street,st_type,from_st,to_st,speedlimit,schoolzone,schoolzo_2,status,workorder,OBJECTID,Join_Count,TARGET_FID,Longitude,Latitude,AADBT,Shape_Leng,geometry
0,0.0,None,None,None,None,15.0,YES,0.0,None,None,1,0,0,0.0,0.0,0.0,0.001610,"LINESTRING (-122.41220 37.75583, -122.41205 37..."
1,107000.0,01ST,ST,HOWARD ST,TEHAMA ST,25.0,None,0.0,None,None,2,0,1,0.0,0.0,0.0,0.000709,"LINESTRING (-122.39603 37.78854, -122.39548 37..."
2,101000.0,01ST,ST,STEVENSON ST,JESSIE ST,25.0,None,0.0,None,None,3,0,2,0.0,0.0,0.0,0.000550,"LINESTRING (-122.39859 37.79057, -122.39816 37..."
3,105000.0,01ST,ST,MINNA ST,NATOMA ST,25.0,None,0.0,None,None,4,0,3,0.0,0.0,0.0,0.000590,"LINESTRING (-122.39703 37.78933, -122.39657 37..."
4,106000.0,01ST,ST,NATOMA ST,HOWARD ST,25.0,None,0.0,None,None,5,0,4,0.0,0.0,0.0,0.000685,"LINESTRING (-122.39657 37.78896, -122.39603 37..."


In [84]:
road_network.crs

<Geographic 2D CRS: GEOGCS["WGS84(DD)",DATUM["WGS84",SPHEROID["WGS84", ...>
Name: WGS84(DD)
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: WGS84
- Ellipsoid: WGS84
- Prime Meridian: Greenwich

In [85]:
road_network1=road_network.drop(['schoolzone',
 'schoolzo_2',
 'status',
 'workorder',
 'OBJECTID',
 'Join_Count',
 'TARGET_FID',
 'Longitude',
 'Latitude'],axis=1)
road_network1.head()
road_network1 = road_network1.rename(columns={'AADBT': 'AADT'})

In [86]:
road_network1['cnn'].nunique()

15684

In [87]:
# Now read the road network with OSM functional class
func=gpd.read_file('H:/mmmiah/Desktop/Bicycle Connectivity Project/SF Validation_data/LTS_data/speed_aadt_func.shp')
func.head()

,cnn,street,st_type,from_st,to_st,speedlimit,schoolzone,schoolzo_2,status,workorder,...,TF_PARK,FT_SEG_STR,FT_INT_STR,TF_SEG_STR,TF_INT_STR,XWALK,Length,Shape_Le_2,Shape_Ar_1,geometry
0,0.0,None,None,None,None,15.0,YES,0.0,None,None,...,0,3.0,0.0,3.0,0.0,0.0,0.087675,0.003611,2.342982e-07,"POLYGON ((-122.41213 37.75584, -122.41198 37.7..."
1,107000.0,01ST,ST,HOWARD ST,TEHAMA ST,25.0,None,0.0,None,None,...,0,4.0,0.0,0.0,0.0,0.0,0.030387,0.001808,9.820555e-08,"POLYGON ((-122.39598 37.78857, -122.39543 37.7..."
2,101000.0,01ST,ST,STEVENSON ST,JESSIE ST,25.0,None,0.0,None,None,...,1,4.0,0.0,0.0,0.0,0.0,0.009157,0.001491,7.888605e-08,"POLYGON ((-122.39854 37.79061, -122.39811 37.7..."
3,105000.0,01ST,ST,MINNA ST,NATOMA ST,25.0,None,0.0,None,None,...,0,4.0,0.0,0.0,0.0,0.0,0.002356,0.001572,8.385906e-08,"POLYGON ((-122.39698 37.78937, -122.39652 37.7..."
4,106000.0,01ST,ST,NATOMA ST,HOWARD ST,25.0,None,0.0,None,None,...,0,4.0,0.0,0.0,0.0,0.0,0.005529,0.001762,9.533942e-08,"POLYGON ((-122.39652 37.78900, -122.39598 37.7..."


In [88]:
func1=func.drop([ 'schoolzone',
 'schoolzo_2',
 'status',
 'workorder',
 'OBJECTID',
 'Join_Count',
 'TARGET_FID',
 'Longitude',
 'Latitude',
 'Shape_Leng',
 'distance',
 'ORIG_FID',
 'Shape_Le_1',
 'Shape_Area',
 'OBJECTID_1',
 'Join_Cou_1',
 'TARGET_F_1', 'SPEED_LIMI', 'FT_BIKE_01','TF_BIKE_01', 'FT_CROSS_L',
 'TF_CROSS_L',
 'TWLTL_CROS',
 'FT_INT_STR',
 'TF_SEG_STR',
 'TF_INT_STR',
 'XWALK',
 'Length',
 'Shape_Le_2',
 'Shape_Ar_1','geometry'],axis=1)
func1.head()
func1 = func1.rename(columns={'AADBT': 'AADT'})

In [89]:
road_network1.shape, func1.shape

((16541, 9), (16572, 22))

In [90]:
# Now combine, aadt+speed+OSM functional class data together
lts_data=pd.merge(road_network1,func1, on=['cnn',
 'st_type',
 'from_st',
 'to_st',
 'speedlimit',
 'AADT'], how='inner')
lts_data.shape

(16541, 25)

In [91]:
lts_data.head()

,cnn,street_x,st_type,from_st,to_st,speedlimit,AADT,Shape_Leng,geometry,street_y,...,INTERSECTI,INTERSE_01,WIDTH,FT_BIKE_IN,TF_BIKE_IN,FT_LANES,TF_LANES,FT_PARK,TF_PARK,FT_SEG_STR
0,0.0,None,None,None,None,15.0,0.0,0.001610,"LINESTRING (-122.41220 37.75583, -122.41205 37...",None,...,99709.0,93444.0,0.0,lane,lane,0.0,0.0,0,0,3.0
1,107000.0,01ST,ST,HOWARD ST,TEHAMA ST,25.0,0.0,0.000709,"LINESTRING (-122.39603 37.78854, -122.39548 37...",01ST,...,89083.0,89080.0,0.0,None,None,4.0,0.0,0,0,4.0
2,101000.0,01ST,ST,STEVENSON ST,JESSIE ST,25.0,0.0,0.000550,"LINESTRING (-122.39859 37.79057, -122.39816 37...",01ST,...,89027.0,89026.0,0.0,None,None,3.0,0.0,1,1,4.0
3,105000.0,01ST,ST,MINNA ST,NATOMA ST,25.0,0.0,0.000590,"LINESTRING (-122.39703 37.78933, -122.39657 37...",01ST,...,89039.0,89041.0,0.0,None,None,4.0,0.0,0,0,4.0
4,106000.0,01ST,ST,NATOMA ST,HOWARD ST,25.0,0.0,0.000685,"LINESTRING (-122.39657 37.78896, -122.39603 37...",01ST,...,89084.0,89083.0,0.0,None,None,4.0,0.0,0,0,4.0


In [92]:
# impute the missing speed limit
lts_data.loc[lts_data['speedlimit']>0].groupby(['FUNCTIONAL'])['speedlimit'].agg(pd.Series.mode)

FUNCTIONAL
living_street     25.0
motorway          99.0
motorway_link     99.0
path              30.0
primary           30.0
primary_link      99.0
residential       15.0
secondary         25.0
secondary_link    15.0
tertiary          25.0
tertiary_link     40.0
trunk             25.0
trunk_link        99.0
unclassified      15.0
Name: speedlimit, dtype: float64

In [93]:
#Define function for speed limit imputation
def speed_fun(FUNCTIONAL):
    if FUNCTIONAL=='living_street':
        return 25
    elif FUNCTIONAL=='motorway':
        return 99
    elif FUNCTIONAL=='motorway_link':
        return 99
    elif FUNCTIONAL=='path':
        return 30
    elif FUNCTIONAL=='primary':
        return 30
    elif FUNCTIONAL=='primary_link':
        return 30
    elif FUNCTIONAL=='residential':
        return 15
    elif FUNCTIONAL=='secondary':
        return 25
    elif FUNCTIONAL=='secondary_link':
        return 15
    elif FUNCTIONAL=='tertiary':
        return 25
    elif FUNCTIONAL=='tertiary_link':
        return 40
    elif FUNCTIONAL=='trunk':
        return 25
    elif FUNCTIONAL=='trunk_link':
        return 25
    elif FUNCTIONAL=='unclassified':
        return 15
    else:
        return 15

In [94]:
# separte the roads that have missing speed
ds=lts_data.loc[lts_data['speedlimit']<=0]
ds['speed']=ds.apply(lambda row:speed_fun(row['FUNCTIONAL']),axis=1)
# assign the speed for the road that have already speed
das=lts_data.loc[lts_data['speedlimit']>0]
das['speed']=das['speedlimit']
# now combine these above two files together
lts1=pd.concat([ds,das], axis=0)
lts1.shape

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


(16541, 26)

# Assign the missing number of lanes for each directions
            lanes:
                - where: "functional_class IN ('primary','secondary')"
                  val: 2
                - where: "functional_class IN ('tertiary')"
                  val: 1
                - where: "functional_class IN ('unclassified','residential','living_street')"
                  val: 0
                - else: 1

In [95]:
def lane_number(FUNCTIONAL):
    if FUNCTIONAL=='primary':
        return 2
    elif FUNCTIONAL=='secondary':
        return 2
    elif FUNCTIONAL=='tertiary':
        return 1
    elif FUNCTIONAL=='unclassified':
        return 1
    elif FUNCTIONAL=='residential':
        return 1
    elif FUNCTIONAL=='living_street':
        return 1
    else:
        return 1

In [96]:
ft_corr=lts1.loc[lts1['FT_LANES']<=0]
ft_corr['forward_lane_num']=ft_corr.apply(lambda row:lane_number(row['FUNCTIONAL']),axis=1 )
ft_non_corr=lts1.loc[lts1['FT_LANES']>0]
ft_non_corr['forward_lane_num']=ft_non_corr['FT_LANES']
ft_corr_al=pd.concat([ft_corr,ft_non_corr])
ft_corr_al.shape

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


(16541, 27)

In [97]:
ft_corr_al.groupby(['FUNCTIONAL'])['forward_lane_num'].count()

FUNCTIONAL
living_street        31
motorway              5
motorway_link        85
path                204
primary             632
primary_link         50
residential       10087
secondary          1869
secondary_link       10
tertiary           2895
tertiary_link         5
trunk                67
trunk_link            6
unclassified        254
Name: forward_lane_num, dtype: int64

In [98]:
tf_corr=ft_corr_al.loc[ft_corr_al['TF_LANES']<=0]
tf_corr['backward_lane_num']=tf_corr.apply(lambda row:lane_number(row['FUNCTIONAL']),axis=1 )
tf_non_corr=ft_corr_al.loc[ft_corr_al['TF_LANES']>0]
tf_non_corr['backward_lane_num']=tf_non_corr['TF_LANES']
tf_corr_al=pd.concat([tf_corr,tf_non_corr])
tf_corr_al.shape

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


(16541, 28)

In [99]:
tf_corr_al.groupby(['backward_lane_num'])['backward_lane_num'].count()

backward_lane_num
1.0    14004
2.0     2438
3.0       99
Name: backward_lane_num, dtype: int64

In [100]:
lts2=tf_corr_al

In [101]:
# we successfuly imputed the speed based city provided shape file, now lets check the lane number,direction and centerline
def direction(forward_lane_num,backward_lane_num):
    if ((forward_lane_num>0)&(backward_lane_num>0)):
        return 2
    else:
        return 1
lts2['direction']=lts2.apply(lambda row:direction(row['forward_lane_num'], row['backward_lane_num']), axis=1) 
lts2.head()    

,cnn,street_x,st_type,from_st,to_st,speedlimit,AADT,Shape_Leng,geometry,street_y,...,TF_BIKE_IN,FT_LANES,TF_LANES,FT_PARK,TF_PARK,FT_SEG_STR,speed,forward_lane_num,backward_lane_num,direction
16,129000.0,02ND,ST,MARKET ST,STEVENSON ST,0.0,16525.99692,0.000722,"LINESTRING (-122.40138 37.78927, -122.40081 37...",02ND,...,sharrow,0.0,0.0,0,0,2.0,25.0,1.0,1.0,2
19,121000.0,02ND,AVE,CORNWALL ST,CLEMENT ST,0.0,0.00000,0.001871,"LINESTRING (-122.46027 37.78501, -122.46014 37...",02ND,...,None,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2
20,122000.0,02ND,AVE,CLEMENT ST,GEARY BLVD,0.0,0.00000,0.001935,"LINESTRING (-122.46014 37.78314, -122.46000 37...",02ND,...,None,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2
22,128000.0,02ND,AVE,HUGO ST,IRVING ST,0.0,39.99996,0.000941,"LINESTRING (-122.45888 37.76530, -122.45882 37...",02ND,...,None,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2
28,124000.0,02ND,AVE,ANZA ST,BALBOA ST,0.0,784.99953,0.001855,"LINESTRING (-122.45986 37.77928, -122.45972 37...",02ND,...,None,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2


In [102]:
lts2.shape

(16541, 29)

# Assign the centerlines based on following assumption
                - where: "functional_class IN ('primary','secondary','tertiary')"
                  val: True
                - where: "functional_class IN ('unclassified','residential','living_street')"
                  val: False
                - else: True

In [103]:
# define centerline
def centerline(FUNCTIONAL):
    if FUNCTIONAL=='primary':
        return 1
    elif FUNCTIONAL=='secondary':
        return 1
    elif FUNCTIONAL=='tertiary':
        return 1
    elif FUNCTIONAL=='unclassified':
        return 0
    elif FUNCTIONAL=='residential':
        return 0   
    elif FUNCTIONAL=='living_street':
        return 0     
    else:
        return 0
lts2['centerline']=lts2.apply(lambda row: centerline(row['FUNCTIONAL']),axis=1)
lts2.head() 

,cnn,street_x,st_type,from_st,to_st,speedlimit,AADT,Shape_Leng,geometry,street_y,...,FT_LANES,TF_LANES,FT_PARK,TF_PARK,FT_SEG_STR,speed,forward_lane_num,backward_lane_num,direction,centerline
16,129000.0,02ND,ST,MARKET ST,STEVENSON ST,0.0,16525.99692,0.000722,"LINESTRING (-122.40138 37.78927, -122.40081 37...",02ND,...,0.0,0.0,0,0,2.0,25.0,1.0,1.0,2,1
19,121000.0,02ND,AVE,CORNWALL ST,CLEMENT ST,0.0,0.00000,0.001871,"LINESTRING (-122.46027 37.78501, -122.46014 37...",02ND,...,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2,0
20,122000.0,02ND,AVE,CLEMENT ST,GEARY BLVD,0.0,0.00000,0.001935,"LINESTRING (-122.46014 37.78314, -122.46000 37...",02ND,...,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2,0
22,128000.0,02ND,AVE,HUGO ST,IRVING ST,0.0,39.99996,0.000941,"LINESTRING (-122.45888 37.76530, -122.45882 37...",02ND,...,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2,0
28,124000.0,02ND,AVE,ANZA ST,BALBOA ST,0.0,784.99953,0.001855,"LINESTRING (-122.45986 37.77928, -122.45972 37...",02ND,...,0.0,0.0,0,0,1.0,15.0,1.0,1.0,2,0


In [104]:
# Assign unique id
lts2['unique_id']=lts2.index


In [105]:
#https://pysal.org/scipy2019-intermediate-gds/deterministic/gds2-rasters.html
import rasterio
from rasterio.plot import show
from rasterio import plot as rioplot
import matplotlib.pyplot as plt

In [106]:
# Open the raster image
barea_slope = rasterio.open('H:/mmmiah/Desktop/Bicycle Connectivity Project/Project-Bicycle Connectivity/Bay Area_LTS/Slope/Slope/cslope11.img')
print(barea_slope.crs)
#rioplot.show(barea_slope)

EPSG:4269


In [107]:
# Reproject the geometries by replacing the values with projected ones
lts2 = lts2.to_crs(epsg=7131)
lts2['length']=lts2.length
lts_new = lts2.to_crs(epsg=4326)
lts_new.head(10)

,cnn,street_x,st_type,from_st,to_st,speedlimit,AADT,Shape_Leng,geometry,street_y,...,FT_PARK,TF_PARK,FT_SEG_STR,speed,forward_lane_num,backward_lane_num,direction,centerline,unique_id,length
16,129000.0,02ND,ST,MARKET ST,STEVENSON ST,0.0,16525.996920,0.000722,"LINESTRING (-122.40138 37.78927, -122.40081 37...",02ND,...,0,0,2.0,25.0,1.0,1.0,2,1,16,70.508916
19,121000.0,02ND,AVE,CORNWALL ST,CLEMENT ST,0.0,0.000000,0.001871,"LINESTRING (-122.46027 37.78501, -122.46014 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,19,207.521292
20,122000.0,02ND,AVE,CLEMENT ST,GEARY BLVD,0.0,0.000000,0.001935,"LINESTRING (-122.46014 37.78314, -122.46000 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,20,214.571476
22,128000.0,02ND,AVE,HUGO ST,IRVING ST,0.0,39.999960,0.000941,"LINESTRING (-122.45888 37.76531, -122.45882 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,22,104.356250
28,124000.0,02ND,AVE,ANZA ST,BALBOA ST,0.0,784.999530,0.001855,"LINESTRING (-122.45986 37.77928, -122.45972 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,28,205.686415
29,123000.0,02ND,AVE,GEARY BLVD,ANZA ST,0.0,0.000000,0.001935,"LINESTRING (-122.46000 37.78121, -122.45986 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,29,214.571424
30,119000.0,02ND,AVE,LAKE ST,CALIFORNIA ST,0.0,0.000000,0.001330,"LINESTRING (-122.46040 37.78686, -122.46031 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,30,147.501201
33,118000.0,02ND,AVE,START: 1-99 BLOCK,LAKE ST,0.0,63.999936,0.000386,"LINESTRING (-122.46043 37.78725, -122.46040 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,33,42.837216
37,125000.0,02ND,AVE,BALBOA ST,CABRILLO ST,0.0,560.999619,0.001869,"LINESTRING (-122.45972 37.77743, -122.45959 37...",02ND,...,0,0,1.0,15.0,1.0,1.0,2,0,37,207.227539
40,130000.0,02ND,ST,STEVENSON ST,JESSIE ST,0.0,16782.998310,0.000597,"LINESTRING (-122.40081 37.78881, -122.40035 37...",02ND,...,0,0,1.0,25.0,1.0,1.0,2,1,40,58.396048


In [108]:
lts_new['unique_id'].nunique()

16541

In [109]:
# get the slope for each link of the network
%time
lts_new
# First identify the starting and ending node of each link
lts_new['starting point'] = None
lts_new['ending point'] = None

for index, row in lts_new.iterrows():
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    lts_new.at[index,'starting point'] = Point(first_coord)
    lts_new.at[index,'ending point'] = Point(last_coord)
    lts_new

CPU times: total: 0 ns
Wall time: 0 ns


In [110]:
# Calculate each link starting lat and long
sf1=lts_new.loc[:,["unique_id","starting point"]]
sf1.columns=["unique_id","geometry"]
sf1['start_lon'] = sf1.geometry.apply(lambda p: p.x)
sf1['start_lat'] = sf1.geometry.apply(lambda p: p.y)
sf2=sf1.loc[:,["unique_id","geometry","start_lon","start_lat"]]
sf2.head(5)

,unique_id,geometry,start_lon,start_lat
16,16,POINT (-122.40137500000002 37.789265),-122.401375,37.789265
19,19,POINT (-122.460266 37.78500699999999),-122.460266,37.785007
20,20,POINT (-122.46014000000001 37.783139999999996),-122.460140,37.783140
22,22,POINT (-122.45888 37.765305000000005),-122.458880,37.765305
28,28,POINT (-122.45985999999999 37.77928),-122.459860,37.779280


In [111]:
sf2.shape

(16541, 4)

In [112]:
# Calculate each link starting elevation
sf2["starting_ele"]= None
#point in test['geometry']:
x = sf2["start_lon"]
y = sf2["start_lat"]
row, col = barea_slope.index(x,y)
sf2["starting_ele"]=barea_slope.read(1)[row,col]

In [113]:
%%time
# Calculate each link end point elevation
ef1=lts_new.loc[:,["unique_id","ending point"]]
ef1.columns=["unique_id","geometry"]
ef1['end_lon'] = ef1.geometry.apply(lambda p: p.x)
ef1['end_lat'] = ef1.geometry.apply(lambda p: p.y)
ef2=ef1.loc[:,["unique_id","geometry","end_lon","end_lat"]]
ef2.head(5)

CPU times: total: 203 ms
Wall time: 207 ms


,unique_id,geometry,end_lon,end_lat
16,16,POINT (-122.40081000000002 37.788815),-122.400810,37.788815
19,19,POINT (-122.46014000000001 37.783139999999996),-122.460140,37.783140
20,20,POINT (-122.46 37.78121),-122.460000,37.781210
22,22,POINT (-122.45882 37.76436600000001),-122.458820,37.764366
28,28,POINT (-122.459724 37.777429999999995),-122.459724,37.777430


In [114]:
ef2['unique_id'].nunique()

16541

In [115]:
%%time
# Calculate the each link end elevation
ef2["ending_ele"]= None
#point in test['geometry']:
x = ef2["end_lon"]
y = ef2["end_lat"]
row, col = barea_slope.index(x,y)
ef2["ending_ele"]=barea_slope.read(1)[row,col]

CPU times: total: 4.45 s
Wall time: 4.45 s


In [116]:
%%time
# Calculate the slope for each link
from functools import reduce
barea_all=[sf2,ef2,lts_new]

barea_all1= reduce(lambda  left,right: pd.merge(left,right,on=['unique_id'],
                                            how='outer'), barea_all).fillna('0')
barea_all1["slope"]=abs((barea_all1['starting_ele']-barea_all1['ending_ele'])/barea_all1['length'])*100
barea_all1=barea_all1.drop_duplicates(subset=['unique_id'], keep='first')
barea_all1.head()

CPU times: total: 125 ms
Wall time: 128 ms


,unique_id,geometry_x,start_lon,start_lat,starting_ele,geometry_y,end_lon,end_lat,ending_ele,cnn,...,FT_SEG_STR,speed,forward_lane_num,backward_lane_num,direction,centerline,length,starting point,ending point,slope
0,16,POINT (-122.40137500000002 37.789265),-122.401375,37.789265,9,POINT (-122.40081000000002 37.788815),-122.400810,37.788815,9,129000.0,...,2.0,25.0,1.0,1.0,2,1,70.508916,POINT (-122.40137500000002 37.789265),POINT (-122.40081000000002 37.788815),0.000000
1,19,POINT (-122.460266 37.78500699999999),-122.460266,37.785007,58,POINT (-122.46014000000001 37.783139999999996),-122.460140,37.783140,59,121000.0,...,1.0,15.0,1.0,1.0,2,0,207.521292,POINT (-122.460266 37.78500699999999),POINT (-122.46014000000001 37.783139999999996),0.481878
2,20,POINT (-122.46014000000001 37.783139999999996),-122.460140,37.783140,59,POINT (-122.46 37.78121),-122.460000,37.781210,60,122000.0,...,1.0,15.0,1.0,1.0,2,0,214.571476,POINT (-122.46014000000001 37.783139999999996),POINT (-122.46 37.78121),0.466045
3,22,POINT (-122.45888 37.765305000000005),-122.458880,37.765305,91,POINT (-122.45882 37.76436600000001),-122.458820,37.764366,97,128000.0,...,1.0,15.0,1.0,1.0,2,0,104.356250,POINT (-122.45888 37.765305000000005),POINT (-122.45882 37.76436600000001),5.749536
4,28,POINT (-122.45985999999999 37.77928),-122.459860,37.779280,62,POINT (-122.459724 37.777429999999995),-122.459724,37.777430,63,124000.0,...,1.0,15.0,1.0,1.0,2,0,205.686415,POINT (-122.45985999999999 37.77928),POINT (-122.459724 37.777429999999995),0.486177


In [117]:
sf2.shape,ef2.shape,lts_new.shape,barea_all1.shape

((16541, 5), (16541, 5), (16541, 34), (16541, 43))

In [118]:
barea_all1.shape

(16541, 43)

In [119]:
barea_all2=barea_all1.drop(['geometry_x',
 'start_lon',
 'start_lat',
 'starting_ele',
 'geometry_y',
 'end_lon',
 'end_lat', 'starting point',
 'ending point'],axis=1)

In [120]:
list(barea_all2)

['unique_id',
 'ending_ele',
 'cnn',
 'street_x',
 'st_type',
 'from_st',
 'to_st',
 'speedlimit',
 'AADT',
 'Shape_Leng',
 'geometry',
 'street_y',
 'ROAD_ID',
 'OSMID',
 'FUNCTIONAL',
 'PATH_ID',
 'ONE_WAY',
 'INTERSECTI',
 'INTERSE_01',
 'WIDTH',
 'FT_BIKE_IN',
 'TF_BIKE_IN',
 'FT_LANES',
 'TF_LANES',
 'FT_PARK',
 'TF_PARK',
 'FT_SEG_STR',
 'speed',
 'forward_lane_num',
 'backward_lane_num',
 'direction',
 'centerline',
 'length',
 'slope']

In [121]:
barea_all2['slope'].describe()

count    1.654100e+04
mean     4.104586e+06
std      1.206194e+08
min      0.000000e+00
25%      1.059085e+00
50%      2.989625e+00
75%      6.538654e+00
max      9.016329e+09
Name: slope, dtype: float64

In [122]:
# Exclusive_bn=bnetw.loc[bnetw['Network_Type']=='Exclusive_Bike_Network']
# Exclusive_bn.shape

In [123]:
# bike_mixed=bnetw.loc[bnetw['Network_Type']=='Bike_Traffic_Mixed_Network']
# bike_mixed.shape

In [124]:
# save this mixed traffic network file
crs = {'init': 'epsg:4326'}
mixtraffic = gpd.GeoDataFrame(barea_all2, crs=crs, geometry = 'geometry')
mixtraffic.to_file("mixed_traffic_network.json", driver="GeoJSON")

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


# Now Prepare Bike lane data

In [125]:
# read the bike network
bnetw=gpd.read_file('H:/mmmiah/Desktop/Bicycle Connectivity Project/SF Validation_data/LTS_data/Bike_network_osm.shp')
bnetw.head()

,objectid,cnn,streetname,from_st,to_st,facility_t,direct,surface_tr,barrier,notes,...,workorder,OBJECTID_3,Join_Cou_2,TARGET_F_2,Longitude,Latitude,AADBT,Shape_Le_2,Shape_Ar_2,geometry
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,None,16,6,480,0.00000,0.00000,0.000000,0.000945,1.991967e-07,"POLYGON ((-122.50279 37.70799, -122.50225 37.7..."
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,None,17,8,474,-122.40112,37.78902,16525.996920,0.000722,1.587010e-07,"POLYGON ((-122.51086 37.78276, -122.51074 37.7..."
2,1688.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,None,18,7,476,-122.39904,37.78741,22482.998460,0.000597,1.359252e-07,"POLYGON ((-122.45376 37.77002, -122.45482 37.7..."
3,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,None,19,6,469,-122.39232,37.78205,3489.999444,0.000692,1.529887e-07,"POLYGON ((-122.50265 37.71867, -122.50267 37.7..."
4,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,None,24,5,489,-122.39320,37.78274,13976.998160,0.000274,7.681173e-08,"POLYGON ((-122.44743 37.77187, -122.44746 37.7..."


In [126]:
ft_corrb=bnetw.loc[bnetw['FT_LANES']<=0]
ft_corrb['forward_lane_num']=ft_corrb.apply(lambda row:lane_number(row['FUNCTIONAL']),axis=1 )
ft_non_corrb=bnetw.loc[bnetw['FT_LANES']>0]
ft_non_corrb['forward_lane_num']=ft_non_corrb['FT_LANES']
bnetw1=pd.concat([ft_corrb,ft_non_corrb])
bnetw1.shape

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


(5408, 88)

In [127]:
# assign the lane number from OSM
tf_corrb=bnetw1.loc[bnetw1['TF_LANES']<=0]
tf_corrb['backward_lane_num']=tf_corrb.apply(lambda row:lane_number(row['FUNCTIONAL']),axis=1 )
tf_non_corrb=bnetw1.loc[bnetw1['TF_LANES']>0]
tf_non_corrb['backward_lane_num']=tf_non_corrb['TF_LANES']
bnetw2=pd.concat([tf_corrb,tf_non_corrb])
bnetw2.shape

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


(5408, 89)

In [128]:
bnetw2['direction']=np.where(bnetw2['direct'].str.contains('2W'),2,1)
bnetw2.head()   

,objectid,cnn,streetname,from_st,to_st,facility_t,direct,surface_tr,barrier,notes,...,TARGET_F_2,Longitude,Latitude,AADBT,Shape_Le_2,Shape_Ar_2,geometry,forward_lane_num,backward_lane_num,direction
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,480,0.00000,0.00000,0.000000,0.000945,1.991967e-07,"POLYGON ((-122.50279 37.70799, -122.50225 37.7...",1.0,1.0,2
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,474,-122.40112,37.78902,16525.996920,0.000722,1.587010e-07,"POLYGON ((-122.51086 37.78276, -122.51074 37.7...",1.0,1.0,2
3,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,469,-122.39232,37.78205,3489.999444,0.000692,1.529887e-07,"POLYGON ((-122.50265 37.71867, -122.50267 37.7...",1.0,1.0,2
4,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,489,-122.39320,37.78274,13976.998160,0.000274,7.681173e-08,"POLYGON ((-122.44743 37.77187, -122.44746 37.7...",1.0,1.0,2
5,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,471,0.00000,0.00000,0.000000,0.001292,2.628030e-07,"POLYGON ((-122.50338 37.71294, -122.50266 37.7...",1.0,1.0,2


In [129]:
bnetw2['direct'].unique()

array(['2W', '1W', None, 'N'], dtype=object)

In [130]:
# separte the roads that have missing speed
dsb=bnetw2.loc[bnetw2['speedlimit']<=0]
dsb['speed']=dsb.apply(lambda row:speed_fun(row['FUNCTIONAL']),axis=1)
# assign the speed for the road that have already speed
dasb=bnetw2.loc[bnetw2['speedlimit']>0]
dasb['speed']=dasb['speedlimit']
# now combine these above two files together
bnetw3=pd.concat([dsb,dasb], axis=0)
bnetw3.shape

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


(5408, 91)

In [131]:
dsb.shape,dasb.shape

((5030, 91), (378, 91))

In [132]:
def network_type(cnn):
    if cnn==0:
        return 'Exclusive_Bike_Network'
    else:
        return 'Bike_Traffic_Mixed_Network'
bnetw3['Network_Type']=bnetw3.apply(lambda row: network_type(row['cnn']),axis=1)
bnetw3.head()

,objectid,cnn,streetname,from_st,to_st,facility_t,direct,surface_tr,barrier,notes,...,Latitude,AADBT,Shape_Le_2,Shape_Ar_2,geometry,forward_lane_num,backward_lane_num,direction,speed,Network_Type
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.00000,0.000000,0.000945,1.991967e-07,"POLYGON ((-122.50279 37.70799, -122.50225 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,37.78902,16525.996920,0.000722,1.587010e-07,"POLYGON ((-122.51086 37.78276, -122.51074 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network
3,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,37.78205,3489.999444,0.000692,1.529887e-07,"POLYGON ((-122.50265 37.71867, -122.50267 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network
4,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,37.78274,13976.998160,0.000274,7.681173e-08,"POLYGON ((-122.44743 37.77187, -122.44746 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network
5,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,0.00000,0.000000,0.001292,2.628030e-07,"POLYGON ((-122.50338 37.71294, -122.50266 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network


In [133]:
bnetw3['symbology'].unique()

array(['BIKE PATH', 'BIKE ROUTE', 'BIKE LANE', 'SEPARATED BIKEWAY',
       'NEIGHBORWAY'], dtype=object)

In [134]:
bnetw3['barrier'].unique()

array([None, 'CONCRETE', 'PARKING; SAFE-HIT POSTS', 'SAFE-HIT POSTS',
       'PARKING', 'CONCRETE; SAFE-HIT POSTS',
       'CONCRETE; PARKING; SAFE-HIT POSTS', 'CONCRETE; PARKING',
       'BACK-IN ANGLED PARKING', 'SAFE-HIT POSTS; PARKING',
       'PARKING; SAFE-HIT POST',
       'PARKING; SAFE-HIT POST; CONCRETE ISLAND',
       'SAFE-HIT POSTS; BIKE SHARE STATION', 'SAFE-HIT POST', 'K-RAIL',
       'PARKING; WALKWAY; SAFE-HIT POSTS',
       'CONCRETE; SAFE-HIT POSTS; BIKE SHARE STATION',
       'PARKING; CONCRETE; SAFE-HIT POSTS', 'SAFE-HIT POSTS; K-RAIL',
       'PARKING; SAFE-HIT POSTS; BIKE SHARE STATIONS',
       'PARKING; CONCRETE ISLAND', 'CURB',
       'CONCRETE ISLAND; SAFE-HIT POSTS',
       'SAFE-HIT POSTS; CONCRETE ISLAND',
       'PARKING; SAFE-HIT POSTS; CONCRETE ISLAND',
       'SAFE HIT POSTS; CONCRETE ISLAND',
       'PARKING; SAFE-HIT POSTS; CONCRETE', 'PARKING;  SAFE-HIT POSTS',
       'SAFE-HIT POSTS; CONCRETE'], dtype=object)

In [135]:
# define Parking lane
bnetw3['parking_lane'] = np.where(bnetw3['barrier'].str.contains('PARKING'),1,0)
bnetw3.head()

,objectid,cnn,streetname,from_st,to_st,facility_t,direct,surface_tr,barrier,notes,...,AADBT,Shape_Le_2,Shape_Ar_2,geometry,forward_lane_num,backward_lane_num,direction,speed,Network_Type,parking_lane
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.000000,0.000945,1.991967e-07,"POLYGON ((-122.50279 37.70799, -122.50225 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,16525.996920,0.000722,1.587010e-07,"POLYGON ((-122.51086 37.78276, -122.51074 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network,0
3,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,3489.999444,0.000692,1.529887e-07,"POLYGON ((-122.50265 37.71867, -122.50267 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0
4,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,13976.998160,0.000274,7.681173e-08,"POLYGON ((-122.44743 37.77187, -122.44746 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network,0
5,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,0.000000,0.001292,2.628030e-07,"POLYGON ((-122.50338 37.71294, -122.50266 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0


In [136]:
# Assign bike lane width
#https://sfgov.org/sfc/bac/bicycle-plan-part-5
def bike_length(symbology):
    if (symbology=='BIKE PATH'):
        return 8
    elif (symbology=='BIKE ROUTE'):
        return 12
    elif (symbology=='BIKE LANE'):
        return 6
#https://dot.ca.gov/-/media/dot-media/programs/design/documents/dib-89-01_kf-a11y.pdf
    elif (symbology=='SEPARATED BIKEWAY'):
        return 7
#https://www.sfbetterstreets.org/design-guidelines/sidewalk-width/#sidewalkWidthTable
    elif (symbology=='NEIGHBORWAY'):
        return 10
    else:
        return 8
bnetw3['bike_lane_width']=bnetw3.apply(lambda row:bike_length(row['symbology']),axis=1)
bnetw3.head()

,objectid,cnn,streetname,from_st,to_st,facility_t,direct,surface_tr,barrier,notes,...,Shape_Le_2,Shape_Ar_2,geometry,forward_lane_num,backward_lane_num,direction,speed,Network_Type,parking_lane,bike_lane_width
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.000945,1.991967e-07,"POLYGON ((-122.50279 37.70799, -122.50225 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0,8
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.000722,1.587010e-07,"POLYGON ((-122.51086 37.78276, -122.51074 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network,0,8
3,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.000692,1.529887e-07,"POLYGON ((-122.50265 37.71867, -122.50267 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0,8
4,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.000274,7.681173e-08,"POLYGON ((-122.44743 37.77187, -122.44746 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network,0,8
5,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,0.001292,2.628030e-07,"POLYGON ((-122.50338 37.71294, -122.50266 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0,8


In [137]:
def parking_lane(parking_lane):
    if parking_lane==1:
        return 8
    else:
        return 0
bnetw3['bike_lane_reach_width']=bnetw3.apply(lambda row:parking_lane(row['parking_lane']),axis=1)
bnetw3.head()

,objectid,cnn,streetname,from_st,to_st,facility_t,direct,surface_tr,barrier,notes,...,Shape_Ar_2,geometry,forward_lane_num,backward_lane_num,direction,speed,Network_Type,parking_lane,bike_lane_width,bike_lane_reach_width
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,1.991967e-07,"POLYGON ((-122.50279 37.70799, -122.50225 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0,8,0
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,1.587010e-07,"POLYGON ((-122.51086 37.78276, -122.51074 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network,0,8,0
3,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,1.529887e-07,"POLYGON ((-122.50265 37.71867, -122.50267 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0,8,0
4,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,7.681173e-08,"POLYGON ((-122.44743 37.77187, -122.44746 37.7...",1.0,1.0,2,30.0,Exclusive_Bike_Network,0,8,0
5,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,2.628030e-07,"POLYGON ((-122.50338 37.71294, -122.50266 37.7...",1.0,1.0,2,15.0,Exclusive_Bike_Network,0,8,0


In [138]:
bnetw3=bnetw3.drop(['geometry'],axis=1)

In [139]:
# read bike polyline shape file
bply=gpd.read_file('H:/mmmiah/Desktop/Bicycle Connectivity Project/SF Validation_data/MTA.bikewaynetwork/geo_export_a1e39e89-170f-4d40-866d-7f03d9a47948.shp')
bply = bply.to_crs(epsg=4326)
bply1=bply.drop(['cnn',
 'streetname',
 'from_st',
 'to_st',
 'facility_t',
 'direct',
 'surface_tr',
 'barrier',
 'notes',
 'dir',
 'double',
 'fy',
 'qtr',
 'street_',
 'length',
 'number_',
 'install_yr',
 'install_mo',
 'update_yr',
 'update_mo',
 'globalid',
 'symbology',
 'sharrow',
 'buffered',
 'raised',
 'contraflow',
 'greenwave',
 'biap',
 'sm_sweeper'],axis=1)

In [140]:
bply.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [141]:
bnetw4=pd.merge(bnetw3, bply, on=['objectid'], how='inner')
bnetw4.head()

,objectid,cnn_x,streetname_x,from_st_x,to_st_x,facility_t_x,direct_x,surface_tr_x,barrier_x,notes_x,...,symbology_y,sharrow_y,buffered_y,raised_y,contraflow_y,greenwave_y,biap_y,sm_sweeper_y,L_mile,geometry
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,BIKE PATH,0.0,NO,NO,NO,NO,NO,NO,0.267051,"LINESTRING (-122.50269 37.70797, -122.50214 37..."
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,BIKE PATH,0.0,NO,NO,NO,NO,NO,NO,0.155166,"LINESTRING (-122.51093 37.78270, -122.51077 37..."
2,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,BIKE PATH,0.0,NO,NO,NO,NO,NO,NO,0.143260,"LINESTRING (-122.50256 37.71871, -122.50258 37..."
3,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,BIKE PATH,0.0,NO,NO,NO,NO,NO,NO,0.056055,"LINESTRING (-122.44733 37.77189, -122.44736 37..."
4,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,BIKE PATH,0.0,NO,NO,NO,NO,NO,NO,0.039456,"LINESTRING (-122.50338 37.71286, -122.50266 37..."


In [142]:
# Now calculate the slope
bnetw4.crs = {'init' :'epsg:4326'}
bnetw4 = gpd.GeoDataFrame(bnetw4, geometry='geometry') 
bnetw4 = bnetw4.to_crs(epsg=7131)
bnetw4['length']=bnetw4.length
bnetw4 = bnetw4.to_crs(epsg=4326)
bnetw4.head(10)

C:\Users\mmmiah\AppData\Local\Temp\ipykernel_17956\3469176368.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  bnetw4.crs = {'init' :'epsg:4326'}


,objectid,cnn_x,streetname_x,from_st_x,to_st_x,facility_t_x,direct_x,surface_tr_x,barrier_x,notes_x,...,sharrow_y,buffered_y,raised_y,contraflow_y,greenwave_y,biap_y,sm_sweeper_y,L_mile,geometry,length
0,1652.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.267051,"LINESTRING (-122.50269 37.70797, -122.50214 37...",429.780898
1,4773.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.155166,"LINESTRING (-122.51093 37.78270, -122.51077 37...",249.717031
2,3319.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.143260,"LINESTRING (-122.50256 37.71871, -122.50258 37...",230.555997
3,583.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.056055,"LINESTRING (-122.44733 37.77189, -122.44736 37...",90.213100
4,3670.0,0.0,None,None,None,CLASS I,2W,None,None,Not City Maintained (OTHER PATH),...,0.0,NO,NO,NO,NO,NO,NO,0.039456,"LINESTRING (-122.50338 37.71286, -122.50266 37...",63.499582
5,3371.0,0.0,None,LINCOLN WAY,MARTIN LUTHER KING JR DR,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.035696,"LINESTRING (-122.47826 37.76591, -122.47833 37...",57.447514
6,4394.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.179803,"LINESTRING (-122.49948 37.71249, -122.50004 37...",289.367486
7,2553.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.164610,"LINESTRING (-122.50010 37.71808, -122.50012 37...",264.916973
8,4228.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.193993,"LINESTRING (-122.50399 37.71688, -122.50397 37...",312.204990
9,864.0,0.0,None,None,None,CLASS I,2W,None,None,None,...,0.0,NO,NO,NO,NO,NO,NO,0.467787,"LINESTRING (-122.44896 37.77227, -122.44890 37...",752.836915


In [143]:
# Assign unique id
bnetw4['unique_id']=bnetw4.index

In [144]:
# get the slope for each link of the network
%time
# First identify the starting and ending node of each link
bnetw4['starting point'] = None
bnetw4['ending point'] = None

for index, row in bnetw4.iterrows():
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    bnetw4.at[index,'starting point'] = Point(first_coord)
    bnetw4.at[index,'ending point'] = Point(last_coord)
    bnetw4

CPU times: total: 0 ns
Wall time: 0 ns


In [145]:
# Calculate each link starting lat and long
sf1b=bnetw4.loc[:,["unique_id","starting point"]]
sf1b.columns=["unique_id","geometry"]
sf1b['start_lon'] = sf1b.geometry.apply(lambda p: p.x)
sf1b['start_lat'] = sf1b.geometry.apply(lambda p: p.y)
sf2b=sf1b.loc[:,["unique_id","geometry","start_lon","start_lat"]]
sf2b.head(5)

,unique_id,geometry,start_lon,start_lat
0,0,POINT (-122.50269000000002 37.70797),-122.502690,37.707970
1,1,POINT (-122.510925 37.78269999999999),-122.510925,37.782700
2,2,POINT (-122.50256 37.71871),-122.502560,37.718710
3,3,POINT (-122.44733 37.77189),-122.447330,37.771890
4,4,POINT (-122.50338000000002 37.712856),-122.503380,37.712856


In [146]:
# Calculate each link starting elevation
sf2b["starting_ele"]= None
#point in test['geometry']:
x = sf2b["start_lon"]
y = sf2b["start_lat"]
row, col = barea_slope.index(x,y)
sf2b["starting_ele"]=barea_slope.read(1)[row,col]

In [147]:
%%time
# Calculate each link end point elevation
ef1b=bnetw4.loc[:,["unique_id","ending point"]]
ef1b.columns=["unique_id","geometry"]
ef1b['end_lon'] = ef1b.geometry.apply(lambda p: p.x)
ef1b['end_lat'] = ef1b.geometry.apply(lambda p: p.y)
ef2b=ef1b.loc[:,["unique_id","geometry","end_lon","end_lat"]]
ef2b.head(5)

CPU times: total: 93.8 ms
Wall time: 90.8 ms


,unique_id,geometry,end_lon,end_lat
0,0,POINT (-122.50229000000002 37.711765),-122.502290,37.711765
1,1,POINT (-122.509476 37.78341),-122.509476,37.783410
2,2,POINT (-122.50235000000002 37.720554),-122.502350,37.720554
3,3,POINT (-122.44737999999998 37.772617),-122.447380,37.772617
4,4,POINT (-122.50265999999999 37.712868),-122.502660,37.712868


In [148]:
%%time
# Calculate the each link end elevation
ef2b["ending_ele"]= None
#point in test['geometry']:
x = ef2b["end_lon"]
y = ef2b["end_lat"]
row, col = barea_slope.index(x,y)
ef2b["ending_ele"]=barea_slope.read(1)[row,col]

CPU times: total: 4.48 s
Wall time: 4.49 s


In [149]:
%%time
# Calculate the slope for each link
from functools import reduce
barea_allb=[sf2b,ef2b,bnetw4]

barea_all1b= reduce(lambda  left,right: pd.merge(left,right,on=['unique_id'],
                                            how='outer'), barea_allb).fillna('0')
barea_all1b["slope"]=abs((barea_all1b['starting_ele']-barea_all1b['ending_ele'])/barea_all1b['length'])*100
barea_all1b=barea_all1b.drop_duplicates(subset=['unique_id'], keep='first')
barea_all1b.head()

CPU times: total: 156 ms
Wall time: 154 ms


<timed exec>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


,unique_id,geometry_x,start_lon,start_lat,starting_ele,geometry_y,end_lon,end_lat,ending_ele,objectid,...,contraflow_y,greenwave_y,biap_y,sm_sweeper_y,L_mile,geometry,length,starting point,ending point,slope
0,0,POINT (-122.50269000000002 37.70797),-122.502690,37.707970,2147483647,POINT (-122.50229000000002 37.711765),-122.502290,37.711765,23,1652.0,...,NO,NO,NO,NO,0.267051,"LINESTRING (-122.50269 37.70797, -122.50214 37...",429.780898,POINT (-122.50269000000002 37.70797),POINT (-122.50229000000002 37.711765),4.996694e+08
1,1,POINT (-122.510925 37.78269999999999),-122.510925,37.782700,70,POINT (-122.509476 37.78341),-122.509476,37.783410,82,4773.0,...,NO,NO,NO,NO,0.155166,"LINESTRING (-122.51093 37.78270, -122.51077 37...",249.717031,POINT (-122.510925 37.78269999999999),POINT (-122.509476 37.78341),4.805439e+00
2,2,POINT (-122.50256 37.71871),-122.502560,37.718710,51,POINT (-122.50235000000002 37.720554),-122.502350,37.720554,43,3319.0,...,NO,NO,NO,NO,0.143260,"LINESTRING (-122.50256 37.71871, -122.50258 37...",230.555997,POINT (-122.50256 37.71871),POINT (-122.50235000000002 37.720554),3.469873e+00
3,3,POINT (-122.44733 37.77189),-122.447330,37.771890,73,POINT (-122.44737999999998 37.772617),-122.447380,37.772617,73,583.0,...,NO,NO,NO,NO,0.056055,"LINESTRING (-122.44733 37.77189, -122.44736 37...",90.213100,POINT (-122.44733 37.77189),POINT (-122.44737999999998 37.772617),0.000000e+00
4,4,POINT (-122.50338000000002 37.712856),-122.503380,37.712856,15,POINT (-122.50265999999999 37.712868),-122.502660,37.712868,33,3670.0,...,NO,NO,NO,NO,0.039456,"LINESTRING (-122.50338 37.71286, -122.50266 37...",63.499582,POINT (-122.50338000000002 37.712856),POINT (-122.50265999999999 37.712868),2.834664e+01


In [150]:
barea_all1b['slope'].describe()

count    5.408000e+03
mean     5.823363e+06
std      1.297451e+08
min      0.000000e+00
25%      7.035734e-01
50%      2.069312e+00
75%      4.197010e+00
max      4.217675e+09
Name: slope, dtype: float64

In [151]:
barea_all2b=barea_all1b.drop(['geometry_x',
 'start_lon',
 'start_lat',
 'starting_ele',
 'geometry_y',
 'end_lon',
 'end_lat', 'starting point',
 'ending point'],axis=1)

In [152]:
barea_all2b['centerline']=barea_all2b.apply(lambda row: centerline(row['FUNCTIONAL']),axis=1)
barea_all2b.head() 

,unique_id,ending_ele,objectid,cnn_x,streetname_x,from_st_x,to_st_x,facility_t_x,direct_x,surface_tr_x,...,raised_y,contraflow_y,greenwave_y,biap_y,sm_sweeper_y,L_mile,geometry,length,slope,centerline
0,0,23,1652.0,0.0,0,0,0,CLASS I,2W,0,...,NO,NO,NO,NO,NO,0.267051,"LINESTRING (-122.50269 37.70797, -122.50214 37...",429.780898,4.996694e+08,0
1,1,82,4773.0,0.0,0,0,0,CLASS I,2W,0,...,NO,NO,NO,NO,NO,0.155166,"LINESTRING (-122.51093 37.78270, -122.51077 37...",249.717031,4.805439e+00,0
2,2,43,3319.0,0.0,0,0,0,CLASS I,2W,0,...,NO,NO,NO,NO,NO,0.143260,"LINESTRING (-122.50256 37.71871, -122.50258 37...",230.555997,3.469873e+00,0
3,3,73,583.0,0.0,0,0,0,CLASS I,2W,0,...,NO,NO,NO,NO,NO,0.056055,"LINESTRING (-122.44733 37.77189, -122.44736 37...",90.213100,0.000000e+00,0
4,4,33,3670.0,0.0,0,0,0,CLASS I,2W,0,...,NO,NO,NO,NO,NO,0.039456,"LINESTRING (-122.50338 37.71286, -122.50266 37...",63.499582,2.834664e+01,0


In [153]:
# save this mixed traffic network file
crs = {'init': 'epsg:4326'}
biken = gpd.GeoDataFrame(barea_all2b, crs=crs, geometry = 'geometry')
biken.to_file("bike_network_aug10.json", driver="GeoJSON")

C:\Users\mmmiah\Anaconda3\envs\geo_env\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


# End of the data processing